This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [2]:
structure_name = "LLZO_010_La_order0_off__Li_111_slab_heavy"

In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_010_La_order0_off__Li_111_slab_heavy
- Initial lattice height: 73.06 Å
- Number of atoms: 996


In [3]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 19.25 Å
Lowest Li slab atom: 38.81 Å
LLZO top z: 34.81 Å
Li penetration into LLZO: -4.00 Å


In [3]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 19.81 Å
Li slab thickness:   19.25 Å
LLZO top z:          34.81 Å
Lowest Li atom z:    38.81 Å
Li penetration into LLZO: -4.00 Å


In [4]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 751 atoms out of 996
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_167692/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [6]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 15:47:46    -2439.515436        3.883504
SciPyFminCG:    1 15:47:50    -2456.116370        2.844161
SciPyFminCG:    2 15:47:55    -2479.030071        2.189074
SciPyFminCG:    3 15:48:00    -2501.377419        1.884531
SciPyFminCG:    4 15:48:04    -2506.587161        2.320475
SciPyFminCG:    5 15:48:07    -2508.593502        1.340279
SciPyFminCG:    6 15:48:10    -2509.449326        1.061292
SciPyFminCG:    7 15:48:15    -2510.251007        0.782037
SciPyFminCG:    8 15:48:19    -2511.021818        0.926525
SciPyFminCG:    9 15:48:22    -2511.984026        1.812991
SciPyFminCG:   10 15:48:26    -2512.788082        1.862383
SciPyFminCG:   11 15:48:29    -2513.975882        1.267841
SciPyFminCG:   12 15:48:34    -2515.067508        1.042040
SciPyFminCG:   13 15:48:38    -2515.776815        1.506717
SciPyFminCG:   14 15:48:41    -2516.713377        1.505249
SciPyFminCG:   15 15:48:45    -2517.157436        1.281138

In [7]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_160920/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 15:55:06    -2566.698077        0.146926
FIRE:    1 15:55:08    -2566.702114        0.121293
FIRE:    2 15:55:10    -2566.708763        0.128965
FIRE:    3 15:55:11    -2566.711613        0.149593
FIRE:    4 15:55:13    -2566.716362        0.161427
FIRE:    5 15:55:15    -2566.721586        0.173094
FIRE:    6 15:55:16    -2566.726336        0.149902
FIRE:    7 15:55:18    -2566.734884        0.108679
FIRE:    8 15:55:20    -2566.743196        0.096343
FIRE:    9 15:55:21    -2566.754356        0.091307
FIRE:   10 15:55:23    -2566.765280        0.122842
FIRE:   11 15:55:25    -2566.781902        0.186422
FIRE:   12 15:55:27    -2566.801374        0.141012
FIRE:   13 15:55:28    -2566.823934        0.106652
FIRE:   14 15:55:30    -2566.849817        0.182550
FIRE:   15 15:55:32    -2566.885912        0.243914
FIRE:   16 15:55:33    -2566.927706        0.293732
FIRE:   17 15:55:35    -2566.961426        0.429017
FIRE:   18 15:

In [8]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [9]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 23.29 Å
Lowest Li slab atom: 34.77 Å
LLZO top z: 34.45 Å
Li penetration into LLZO: -0.32 Å


In [5]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)

             Step     Time          Energy          fmax
SciPyFminCG:    0 16:45:03    -2567.417358        3.865265
SciPyFminCG:    1 16:45:07    -2593.323830        4.884127
SciPyFminCG:    2 16:45:10    -2619.233625        5.405836
SciPyFminCG:    3 16:45:13    -2649.069428        5.895391
SciPyFminCG:    4 16:45:17    -2673.479633        5.511427
SciPyFminCG:    5 16:45:20    -2696.921219        3.401116
SciPyFminCG:    6 16:45:23    -2712.229156        2.519426
SciPyFminCG:    7 16:45:26    -2720.459215        3.598322
SciPyFminCG:    8 16:45:29    -2730.370050        3.113131
SciPyFminCG:    9 16:45:33    -2740.961935        4.450985
SciPyFminCG:   10 16:45:34    -2745.326065        3.581830
SciPyFminCG:   11 16:45:37    -2753.624039        2.391411
SciPyFminCG:   12 16:45:40    -2757.822893        1.691708
SciPyFminCG:   13 16:45:44    -2761.221491        2.470535
SciPyFminCG:   14 16:45:47    -2764.899822        1.550924
SciPyFminCG:   15 16:45:50    -2768.414065        1.736087

In [6]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_167692/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 17:03:51    -2955.874323        0.251994
FIRE:    1 17:03:53    -2955.887146        0.206771


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.281904852979847e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:    2 17:03:54    -2955.903056        0.172076


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.35358779783127e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


...

FIRE:  398 17:12:13    -2959.177222        0.211638


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.3113848503046437e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  399 17:12:15    -2959.177222        0.187024


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 3.3715061653558155e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  400 17:12:16    -2959.176985        0.152636


In [ ]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_010_La_order0_off__Li_111_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


: 